In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
repo_dir = '/home/labs/amit/noamsh/repos/MM_2023'
sys.path.append(repo_dir)

In [ ]:
from pathlib import Path
import os

from omegaconf import OmegaConf
import scanpy as sc
import numpy as np
import pandas as pd

import anndata as ad
import scvi
import pyreadr

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier, XGBRFClassifier

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

from evaluation.visualizations import make_confusion_matrix
from data_loading.utils import load_dataframe_from_file, merge_labels_to_adata
from sc_classification.var_genes import normalize_and_choose_genes, shuang_genes_to_keep
from io_utils import generate_path_in_output_dir

In [ ]:
config_path = Path(repo_dir, 'config.yaml')
conf = OmegaConf.load(config_path)
update_results = False

In [ ]:
from datetime import date
ts_iso = date.today().isoformat()

sc.set_figure_params(dpi=150, dpi_save=300)
figures_dir = Path(conf.outputs.output_dir, "figures", ts_iso)
update_figures = True

In [ ]:
# load_ts_iso = "2024-05-19"
# data_version = "20240515"

# load_ts_iso = "2024-05-21"
# data_version = "20240519"

load_ts_iso = "2024-08-18"
data_version = "20240813"

adata_for_clustering = ad.read_h5ad(generate_path_in_output_dir(conf, conf.outputs.inferred_missing_annotation_file_name,
                                                 with_version=data_version, with_date_timestamp=load_ts_iso))
# adata_with_annot_and_scvi_path = Path(conf.outputs.output_dir, f"adata_with_scvi_annot_pred_data_v_{data_version}_ts_{load_ts_iso}.h5ad")
# adata_for_clustering = ad.read_h5ad(adata_with_annot_and_scvi_path)
adata_for_clustering

In [ ]:
super_pop = conf.annotation.major_cell_type_column
SCVI_LATENT_KEY = conf.scvi_settings.scvi_latent_key
label_col = conf.annotation.cell_type_column

In [ ]:
from pre_processing.utils import count_number_of_annotation_in_neighborhood

In [ ]:
count_number_of_annotation_in_neighborhood(adata_for_clustering, super_pop, "PC")
count_number_of_annotation_in_neighborhood(adata_for_clustering, super_pop, "CD45")

In [ ]:
adata_for_clustering[adata_for_clustering.obs[super_pop]=="PC"].obs['count_of_PC_in_neighborhood'].hist(bins=125)
adata_for_clustering[adata_for_clustering.obs[super_pop]=="PC"].obs['count_of_CD45_in_neighborhood'].hist(bins=125)

In [ ]:
adata_for_clustering.obs["bad_pcs"] = (adata_for_clustering.obs[super_pop]=="PC") & (adata_for_clustering.obs['count_of_PC_in_neighborhood'] < adata_for_clustering.obs['count_of_CD45_in_neighborhood'])
# adata_for_clustering.obs["bad_pcs"] = adata_for_clustering.obs["bad_pcs"].astype('category')

In [ ]:
adata_for_clustering.obs["bad_pcs"].sum()

In [ ]:
sc.pl.umap(adata_for_clustering, color=['count_of_PC_in_neighborhood', super_pop,'leiden', label_col], ncols=2, legend_loc="on data")
sc.pl.umap(adata_for_clustering[adata_for_clustering.obs["bad_pcs"]], color=['Method'], ncols=1, size=50) #, legend_loc="on data"


In [ ]:
cluster_annot = {}
for c, group in adata_for_clustering.obs[["leiden", super_pop]].groupby("leiden"):
    cluster_annot[c] = group.value_counts().index[0][1]
# cluster_annot

In [ ]:
adata_for_clustering.obs["bad_pcs_leiden"] = (adata_for_clustering.obs[super_pop]=="PC") & (adata_for_clustering.obs['leiden'].apply(lambda x: cluster_annot[x]=="CD45"))
adata_for_clustering.obs["bad_pcs_leiden"].sum()

In [ ]:
sc.pl.umap(adata_for_clustering[adata_for_clustering.obs["bad_pcs_leiden"]], color=['Method'], ncols=1, size=50)#  , legend_loc="on data"

In [ ]:
adata_for_clustering.obs["bad_pcs_all"] = ((adata_for_clustering.obs["bad_pcs_leiden"]) | (adata_for_clustering.obs["bad_pcs"]))
adata_for_clustering.obs["bad_pcs_all"].sum()

In [ ]:
pc_close_other_pc_col = "pc_close_other_pc"
adata_for_clustering.obs[pc_close_other_pc_col] = (adata_for_clustering.obs[super_pop] =='PC') & (~ adata_for_clustering.obs["bad_pcs_all"])
adata_for_clustering.obs[pc_close_other_pc_col] = adata_for_clustering.obs[pc_close_other_pc_col].astype("category")

In [ ]:
sc.pl.umap(adata_for_clustering, color=['Method', super_pop, label_col, 'leiden'], ncols=1, legend_loc="on data")

#### view PC UMAP

In [ ]:
sc.pl.umap(adata_for_clustering[adata_for_clustering.obs[pc_close_other_pc_col]==True], color=[label_col, 'Method','Disease'], ncols=2, palette="Set1")

# pc adata

## load - only pc adata

In [ ]:
# load_ts_iso = "2024-05-21"
# data_version = "20240519"

# load_ts_iso = "2024-06-20"
# data_version = "20240619"

adata_pc_with_annot_and_scvi_path = Path(conf.outputs.output_dir, f"adata_with_scvi_annot_pred_data_v_{data_version}_ts_{load_ts_iso}_only_pc.h5ad")
adata_only_pc = ad.read_h5ad(adata_pc_with_annot_and_scvi_path)
adata_only_pc

## view only pc neigborhood

In [ ]:
from clinical_predictions.clinical_data_loading import load_and_process_clinical_data
from data_loading.utils import get_updated_disease_col

new_hospital_path = Path('/home/labs/amit/noamsh/data/mm_2023/clinical_prediction/Anonymized_CRF_BP_01082024.xlsx')
clinical_data_df = load_and_process_clinical_data(new_hospital_path, code_lower_case=True, get_hospital_stage=True, 
                                                  get_post_treatment=False, get_treatment_history=False)
clinical_data_df['Disease Stage Hospital'].value_counts()


In [ ]:
adata_only_pc.obs['Hospital.Code'] = adata_only_pc.obs['Hospital.Code'].str.lower()
adata_only_pc.obs['Biopsy.Sequence'] = adata_only_pc.obs['Biopsy.Sequence'].astype(int)
merged = adata_only_pc.obs.merge(clinical_data_df, how='left', left_on=['Hospital.Code', 'Biopsy.Sequence'],
                                       right_on=['Code', 'Biopsy sequence No.'])

merged['updated_Disease'] = get_updated_disease_col(merged, 
                      disease_col='Disease', hospital_disease_col='Disease Stage Hospital',
                      update_non_naive_NDMM=False, remove_PRMM=True)

merged.index = adata_only_pc.obs_names
adata_only_pc.obs['Disease'] = merged['updated_Disease']

In [ ]:
adata_only_pc.obs["MGUS"] = (adata_only_pc.obs['Disease'] == "MGUS").astype("category")
adata_only_pc.obs["SMM"] = (adata_only_pc.obs['Disease'] == "SMM").astype("category")
adata_only_pc.obs["NDMM"] = (adata_only_pc.obs['Disease'] == "NDMM").astype("category")
adata_only_pc.obs["non_naive_NDMM"] = (adata_only_pc.obs['Disease'] == "non_naive_NDMM").astype("category")
sc.pl.umap(adata_only_pc, color=['MGUS', 'SMM', "NDMM", "non_naive_NDMM"], ncols=3, palette="Paired")

In [ ]:
if update_figures:
    with plt.rc_context():  # Use this to set figure params like size and dpi
        sc.pl.umap(adata_only_pc, color=['Disease', 'Method', 'Tissue', 'pc_with_tme_environment'], ncols=2, palette="Paired", show=False)
        plt.savefig(Path(figures_dir, "pc_umap_annotation_method.pdf"), bbox_inches="tight", format="pdf")
        
else:
    sc.pl.umap(adata_only_pc, color=['Cohort', 'Method','Tissue','pc_with_tme_environment'], ncols=2, palette="Paired")

In [ ]:
small_clusters = adata_only_pc.obs['leiden'].value_counts()[adata_only_pc.obs['leiden'].value_counts() < 20]
adata_only_pc.obs['leiden'] = adata_only_pc.obs['leiden'].apply(lambda x: np.nan if x in small_clusters.index else x)

In [ ]:
if update_figures:
    with plt.rc_context():  # Use this to set figure params like size and dpi
        sc.pl.umap(adata_only_pc, color=['leiden'], ncols=1, legend_loc='on data', palette="plasma", show=False) # hsv
        plt.savefig(Path(figures_dir, "pc_umap_leiden.pdf"), bbox_inches="tight", format="pdf")
        
else:
    sc.pl.umap(adata_only_pc, color=['leiden'], ncols=1, legend_loc='on data', palette="plasma")


In [ ]:
healthy_clusters = ['0','3', '4']
healthy_like_clusters = ['6', '25']

def _transform_clusters_to_pc_annotation(leidein_caluster):
    if leidein_caluster in healthy_clusters:
        return "Healthy"
    if leidein_caluster in healthy_like_clusters:
        return "Healthy_Like"
    else:
        return "Malignant"
        
adata_only_pc.obs["pc_annotation"] = adata_only_pc.obs["leiden"].apply(_transform_clusters_to_pc_annotation)

if update_figures:
    with plt.rc_context():  # Use this to set figure params like size and dpi
        sc.pl.umap(adata_only_pc, color=['pc_annotation'], ncols=1, legend_loc='on data', palette="Paired", show=False) 
        plt.savefig(Path(figures_dir, "pc_umap_healthy_annotation.pdf"), bbox_inches="tight", format="pdf")
        
else:
    sc.pl.umap(adata_only_pc, color=['pc_annotation'], ncols=1, legend_loc='on data', palette="Paired")


In [ ]:
genes_of_interst = ["CD38", "TXNIP", "FCRL5", "SSR4", "SDC1", "TNFRSF17", "DERL3", "MZB1", "CCND1", "FRZB", "LAMP5", "NSD2", "FGFR3", "ITGB7", "CDR1", "SPP1", "PPIA", "GPRC5D", "CCND2"]

if update_figures:
    with plt.rc_context():  # Use this to set figure params like size and dpi
        sc.pl.umap(adata_only_pc, color=genes_of_interst, ncols=3, show=False)
        plt.savefig(Path(figures_dir, "pc_umap_marker_genes.pdf"), bbox_inches="tight", format="pdf")    
else:
    sc.pl.umap(adata_only_pc, color=genes_of_interst, ncols=3)

In [ ]:
# number of umis to consider noisy malignant
np.exp(6.5)

In [ ]:
cell_patients_count = adata_only_pc.obs["number_of_diffrent_patients_in_nighborhood"]
adata_only_pc.obs["one_pateint_in_nighborhood"] = (cell_patients_count == 1).astype("category")
adata_only_pc.obs["small_pateint_in_nighborhood"] = (cell_patients_count.apply(lambda x: x in (1,2,3))).astype("category")
adata_only_pc.obs['log_total_counts'] = np.log(adata_only_pc.obs['total_counts'])
adata_only_pc.obs['noisy_malignant'] = ((adata_only_pc.obs["number_of_diffrent_patients_in_nighborhood"] > 15) & 
                                        (adata_only_pc.obs['pc_annotation']!="Healthy")).astype("category")
adata_only_pc.obs['noisy_malignant_by_umi'] = ((adata_only_pc.obs["log_total_counts"] < 6.5) & 
                                        (adata_only_pc.obs['pc_annotation']!="Healthy")).astype("category")


In [ ]:
if update_figures:
    with plt.rc_context():  # Use this to set figure params like size and dpi
        sc.pl.umap(adata_only_pc,
               color=["number_of_diffrent_patients_in_nighborhood", "one_pateint_in_nighborhood", "small_pateint_in_nighborhood", 'log_total_counts'],
               ncols=2 ,color_map="magma", palette='Paired', show=False)
        plt.savefig(Path(figures_dir, "pc_umap_patients_in_neigborhood.pdf"), bbox_inches="tight", format="pdf")    
else:
    sc.pl.umap(adata_only_pc,
               color=["number_of_diffrent_patients_in_nighborhood", "one_pateint_in_nighborhood", "small_pateint_in_nighborhood", 'log_total_counts',
                        'pct_counts_mt', 'noisy_malignant', 'noisy_malignant_by_umi'],
               ncols=2 ,color_map="magma", palette='Paired')

In [ ]:
df = adata_only_pc.obs
fig = px.violin(df, y="number_of_diffrent_patients_in_nighborhood",x="pc_annotation", box=True)
fig.show()
# fig = px.violin(df, y="n_genes_by_counts",x="noisy_malignant", box=True)
# fig.show()

In [ ]:
rows = []
count_thesholds = [400, 450, 500, 550, 600, 650, 700]
n_mal_per_patient = [20, 30, 40, 50, 70, 100]
for thresh in count_thesholds:
    _adata = adata_only_pc[adata_only_pc.obs["total_counts"] >= thresh]
    _mal_adata = _adata[_adata.obs['pc_annotation']!="Healthy"]
    for n_mal_thresh in n_mal_per_patient:
        n_patients_by_malignat_pcs = (_mal_adata.obs['Hospital.Code'].value_counts() >= n_mal_thresh).sum()
        rows.append((n_patients_by_malignat_pcs, thresh, n_mal_thresh))

In [ ]:
df = pd.DataFrame(rows, columns=["n_patients", "cell_thresh_count", "n_malignant_per_patient"])
px.line(df, x="cell_thresh_count", y="n_patients", color="n_malignant_per_patient")

In [ ]:
non_noisy_malignant = adata_only_pc[adata_only_pc.obs['noisy_malignant_by_umi']==False]
non_noisy_malignant

In [ ]:
only_pc_neighbors_key = "PC_nighbors"
sc.pp.neighbors(non_noisy_malignant, use_rep='X_scVI', n_neighbors=30, key_added=only_pc_neighbors_key)
sc.tl.umap(non_noisy_malignant, min_dist=0.3, neighbors_key=only_pc_neighbors_key)

In [ ]:
sc.pl.umap(non_noisy_malignant,
               color=["number_of_diffrent_patients_in_nighborhood", "one_pateint_in_nighborhood", "small_pateint_in_nighborhood", 'pc_annotation'],
               ncols=2 ,color_map="magma", palette='Paired')

In [ ]:
if update_figures:
    with plt.rc_context():  # Use this to set figure params like size and dpi
        sc.pl.umap(non_noisy_malignant, color=genes_of_interst, ncols=3, show=False)
        plt.savefig(Path(figures_dir, "pc_filtered_umap_marker_genes.pdf"), bbox_inches="tight", format="pdf")    
else:
    sc.pl.umap(non_noisy_malignant, color=genes_of_interst, ncols=3)

In [ ]:
non_noisy_malignant.obs.groupby('Hospital.Code')['Disease'].nunique().value_counts()

### save

In [ ]:
data_version, load_ts_iso

In [ ]:
annotated_only_pc_path = Path(conf.outputs.output_dir, f"adata_with_scvi_annot_pred_data_v_{data_version}_ts_{load_ts_iso}_only_pc_annotated.h5ad")
annotated_filtered_only_pc_path = Path(conf.outputs.output_dir, f"adata_with_scvi_annot_pred_data_v_{data_version}_ts_{load_ts_iso}_only_pc_annotated_filtered.h5ad")

adata_only_pc.write(annotated_only_pc_path)
non_noisy_malignant.write(annotated_filtered_only_pc_path)

In [ ]:
annotated_only_pc_path = Path(conf.outputs.output_dir, f"adata_with_scvi_annot_pred_data_v_{data_version}_ts_{load_ts_iso}_only_pc_annotated.h5ad")
annotated_filtered_only_pc_path = Path(conf.outputs.output_dir, f"adata_with_scvi_annot_pred_data_v_{data_version}_ts_{load_ts_iso}_only_pc_annotated_filtered.h5ad")

adata_only_pc = ad.read_h5ad(annotated_only_pc_path)
non_noisy_malignant = ad.read_h5ad(annotated_filtered_only_pc_path)


In [ ]:
Diseases = non_noisy_malignant.obs['Disease'].unique()
for d in Diseases:
    non_noisy_malignant.obs[d] = (non_noisy_malignant.obs['Disease'] == d).astype("category")

sc.pl.umap(non_noisy_malignant, color=Diseases, ncols=2, palette="Paired")

In [ ]:
non_noisy_malignant.obs["Hospital.Code"]

In [ ]:
from sc_classification.train_test_split import split_adata
patrint_id_col = "Hospital.Code"
train_adata, test_adata = split_adata(adata_only_pc, split_by_obs_column=patrint_id_col, seed=42)

In [ ]:
label_col = "pc_annotation"
train_adata.obs[label_col].value_counts()

In [ ]:
train_X = train_adata.obsm[SCVI_LATENT_KEY]
test_X = test_adata.obsm[SCVI_LATENT_KEY]

class_map = {'Healthy':1, 'Healthy_Like':1, 'Malignant':0}

train_y = train_adata.obs[label_col].map(class_map)
test_y = test_adata.obs[label_col].map(class_map)

In [ ]:
model = KNeighborsClassifier(n_neighbors=35)
# model = SVC(C=1.0, gamma='auto')
model = XGBRFClassifier(max_depth=6, n_estimators=256)
model.fit(train_X, train_y)
test_y_pred = model.predict(test_X)
train_y_pred = model.predict(train_X)
model

In [ ]:
print("train")
cats = list(model.classes_)
cm = confusion_matrix(train_y, train_y_pred, labels=cats)
make_confusion_matrix(cm , categories=cats, figsize=(12, 9))

print("test")
cats = list(model.classes_)
cm = confusion_matrix(test_y, test_y_pred, labels=cats)
make_confusion_matrix(cm , categories=cats, figsize=(12, 9))

In [ ]:
mars_adata = adata_only_pc[adata_only_pc.obs['Method']=="MARS"]
spid_adata = adata_only_pc[adata_only_pc.obs['Method']=="SPID"]

mars_X = mars_adata.obsm[SCVI_LATENT_KEY]
spid_X = spid_adata.obsm[SCVI_LATENT_KEY]

mars_y = mars_adata.obs[label_col]
spid_y = spid_adata.obs[label_col]

In [ ]:
mars_model = KNeighborsClassifier(n_neighbors=35)
mars_model.fit(mars_X, mars_y)

In [ ]:
spid_y_pred = mars_model.predict(spid_X)

In [ ]:
cats = list(mars_model.classes_)
cm = confusion_matrix(spid_y, spid_y_pred, labels=cats)
make_confusion_matrix(cm , categories=cats, figsize=(12, 9))

In [ ]:
spid_model = KNeighborsClassifier(n_neighbors=35)
spid_model.fit(spid_X, spid_y)

In [ ]:
mars_y_pred = spid_model.predict(mars_X)

In [ ]:
cats = list(spid_model.classes_)
cm = confusion_matrix(mars_y, mars_y_pred, labels=cats)
make_confusion_matrix(cm , categories=cats, figsize=(14, 10))

#### healthy tumor classification

In [ ]:

train_y_pred = pd.Series(index=train_y.index, data=train_y_pred)
test_y_pred = pd.Series(index=test_y.index, data=test_y_pred)
# pd.concat([test_y_pred, train_y_pred]).astype("category")

In [ ]:
pred_col = "pred" 
adata_only_pc.obs[pred_col] = pd.concat([test_y_pred, train_y_pred]).astype("category")
adata_only_pc.obs["false_healty"] = ((adata_only_pc.obs[pred_col] == 1) & (adata_only_pc.obs[label_col] == 'Malignant')).astype("category")
adata_only_pc.obs["false_malignant"] = ((adata_only_pc.obs[pred_col] == 0) & (adata_only_pc.obs[label_col] == 'Healthy')).astype("category")

In [ ]:
sc.pl.umap(adata_only_pc, color=[label_col, pred_col , "false_healty", "false_malignant"], ncols=2 , palette='Paired') # palette='tab2'

In [ ]:
# sc.tl.embedding_density(adata_only_pc, basis='umap', groupby='false_healty')
# sc.pl.embedding_density(adata_only_pc, basis="umap", key='umap_density_false_healty')

# evaluation of gene expession

In [ ]:
sc.tl.rank_genes_groups(adata, 'pc_annotation', method='logreg')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)